## Comparison: a-Priori SBM Fitting with Different Groupings ##

It is often desirable to create models of data so we can generate new samples, learn something about the data' structure,
or for use in some larger analysis. SBM models define different connection probabilities for different communities in
a graph.

Here we enforce a community structure a-priori based a some connectome attributes or combinations thereof, and fit an SBM.
In this case, the connection probability between two blocks is simply the total number of observed connections between them,
divided by the total number of possible connections between them.

We fit these SBMs using different attributes or combinations of attributes from the connectome, and then compute the
BIC (Bayesian Information Criterion) for each model. This gives us na idea of which attributes best explain the topological
structure of the connectome.

For consistency, we use the attributes `hemisphere` (the side of the brain a neuron is on), `cell_type` (the high-level
classification of a neuron as interneuron, sensory, or motor) and their combination as groups for fitting. The analysis
is only preformed on the Ciona Intestinalis Connectome, two C. Elegans Worm Wiring Connectomes, and the 8 C. Elegans
Developmental Connectomes, because only these have both these attributes defined.

*Note that the code to do the actual SBM fitting is located in `apriori_sbms.py`*

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
sys.path.append("../")
import graspologic as gr
from graspologic import layouts
import numpy as np
import pandas as pd
import networkx as nx
from graph import GraphIO
from apriori_sbm_code import fit_apriori_sbm

## Ciona Intestinalis ##
We'll start with the single Ciona Connectome. First we load the data:

In [ ]:
path = '../json_connectomes/ciona.json'
ciona_connectome, _, _, _ = GraphIO.load(path)

Now we'll fit an SBM for each grouping

In [ ]:
# In Ciona, Hemisphere is called side and cell_type is called annotation
groups = [['Side'], ['Annotation'], ['Side', 'Annotation']]
rows = {'Ciona Intestinalis': {}}
for group in groups:
    rows['Ciona Intestinalis'][str(group)] = fit_apriori_sbm(ciona_connectome, group, plot_sbms=False)
ciona_results = pd.DataFrame.from_dict({(i,j): rows[i][j]
                           for i in rows.keys()
                           for j in rows[i].keys()},
                       orient='index')
ciona_results.head()

Great, now lets plot the BIC for different groupings:

In [ ]:
fig, ax = plt.subplots(2, 1)
out_index = ciona_results.index.levels[0]
for i, key in enumerate(out_index):
    data = ciona_results.loc[key]
    ax[0].scatter(data['bic'].index, data['bic'], label=key)
    ax[1].scatter(data['liklihood'].index, data['liklihood'], label=key)
    ax[0].set(xlabel='Group', ylabel='-BIC')
    ax[1].set(xlabel='Group', ylabel='Likelihood')
fig.suptitle('BIC and Likelihood: Witvilet at Different Developmental Stages')

Interesting, it appears annotation has the best BIC but the worst liklihood. This is
probably because annotation has much fewer parameters than the other groupings. (For Ciona
'Side' is broken into 11 groupings instead of just 'Left' and 'Right'). Therefore, based on
the BIC, annotation explains the structure better, and the other groupings are likely
fitting noise.

## C. Elegans Developmental Conectomes (Witviliet) ##

We will preform this analysis for all 8 developmental stages at once.

In [ ]:
# load some elegans witvilet data
base = '../c_elegans/witvilet2020/graphs/'
graph_files = sorted(os.listdir(base))
wit_connectomes = []
for f in graph_files:
    # These contain multiple synapse types, and thus can have multiedges and need to be flattened.
    # Alternatively, we could preform the analysis separately for each synapse type
    wit_connectomes.append(GraphIO.flatten_multigraph(GraphIO.load(os.path.join(base, f))[0]))

Now we will fit sbms for each grouping for each connectome.

In [ ]:
groups = [["hemisphere"], ["cell_type0"], ["hemisphere", "cell_type0"]]
rows = {}
for i in range(len(wit_connectomes)):
    g_row = {}
    for group in groups:
        g_row[str(group)] = fit_apriori_sbm(wit_connectomes[i], group, plot_sbms=False)
    rows['witvilet_' + str(i)] = g_row
wit_results = pd.DataFrame.from_dict({(i,j): rows[i][j]
                           for i in rows.keys()
                           for j in rows[i].keys()},
                       orient='index')
wit_results.head(24)


Let's plot the BIC and Likelihood

In [ ]:
fig, ax = plt.subplots(8, 2)
out_index = wit_results.index.levels[0]
colors = ['red', 'blue', 'green', 'orange', 'purple', 'grey', 'cyan', 'pink']
for i, key in enumerate(out_index):
    data = wit_results.loc[key]
    ax[i, 0].scatter(data['bic'].index, data['bic'], label=key, color=colors[i])
    ax[i, 1].scatter(data['liklihood'].index, data['liklihood'], label=key, color=colors[i])
    ax[i, 0].set(xlabel='Groups, stage ' + str(i), ylabel='-BIC')
    ax[i, 1].set(xlabel='Groups, stage ' + str(i), ylabel='Likelihood')
plt.ylabel('-BIC \t Likelihood')
fig.suptitle('BIC and Likelihood: C. Elegans at Different Developmental Stages')
fig.set_size_inches(16, 32)

Once again, we find that cell type explains the topology the best. This is still not entirely surprising, since the
connectivity of interneurons should be very different than that of sensory or motor neurons

## C. Elegans Male and Hermaphrodite Conectomes (Worm Wiring) ##

In [ ]:
# load chemical synapse connectome files
path_h = '../c_elegans/worm_wiring/graphs/connectome/Hermaphrodite/0.json'
path_m = '../c_elegans/worm_wiring/graphs/connectome/Male/0.json'
ww_connectomes = []
ww_connectomes.append(GraphIO.load(path_h)[0])
ww_connectomes.append(GraphIO.load(path_m)[0])

In [ ]:
groups = [["hemisphere"], ["cell_type0"], ["hemisphere", "cell_type0"]]
types = ['Hermaphrodite', 'Male']
rows = {}
for i in range(len(ww_connectomes)):
    g_row = {}
    for group in groups:
        g_row[str(group)] = fit_apriori_sbm(ww_connectomes[i], group, plot_sbms=False)
    rows['C. Elegans ' + types[i]] = g_row
ww_results = pd.DataFrame.from_dict({(i,j): rows[i][j]
                           for i in rows.keys()
                           for j in rows[i].keys()},
                       orient='index')
ww_results.head(6)

One last time, we'll plot BIC and Likelihood

In [ ]:
fig, ax = plt.subplots(2, 2)
out_index = ww_results.index.levels[0]
colors = ['red', 'blue']
for i, key in enumerate(out_index):
    data = ww_results.loc[key]
    ax[i, 0].scatter(data['bic'].index, data['bic'], label=key, color=colors[i])
    ax[i, 1].scatter(data['liklihood'].index, data['liklihood'], label=key, color=colors[i])
    ax[i, 0].set(xlabel='Groups, ' + types[i], ylabel='-BIC')
    ax[i, 1].set(xlabel='Groups, ' + types[i], ylabel='Likelihood')
plt.ylabel('-BIC \t Likelihood')
fig.suptitle('BIC and Likelihood: Worm Wiring C. Elegans Hermaphrodite and Male')
fig.set_size_inches(16, 8)

For a third time, cell type groupings get the best BIC, whcih makes sense since this is the same species as the developmental data
